<a href="https://colab.research.google.com/github/spegas/Dacon/blob/main/Get_Price_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=0aac2173c28b79e092605c1f3de19f892fa6b78d1fd1a7c2d53fbe7a04fe7ab3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [9]:
import pandas as pd
import os
import FinanceDataReader as fdr
import wget

In [7]:
if not os.path.isdir("base"):
  print("base 디렉토리 생성")
  os.makedirs("base")

if not os.path.isdir("price"):
  print("price 디렉토리 생성")
  os.makedirs("price")

In [10]:
stock_list_file_url = "https://raw.githubusercontent.com/spegas/Dacon/main/stock_list.csv"
if os.path.exists('base/stock_list.csv'):
  os.remove('base/stock_list.csv')
  print('이전에 저장된 주식 종목 리스트 파일을 삭제 합니다.')
wget.download(stock_list_file_url, 'base/stock_list.csv')
print('주식 종목 리스트 파일 다운로드 완료')

주식 종목 리스트 파일 다운로드 완료


In [13]:
stock_list_name = 'base/stock_list.csv'
pd_stock_list = pd.read_csv(stock_list_name)
pd_stock_list['종목코드'] = pd_stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
pd_stock_list.head(10)

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
5,삼성전자우,005935,KOSPI
6,LG화학,051910,KOSPI
7,삼성SDI,006400,KOSPI
8,현대차,005380,KOSPI
9,셀트리온,068270,KOSPI


In [14]:
start_date = '20200101'
end_date = '20211031'

In [15]:
for f_index in range(0, pd_stock_list.shape[0]):
  f_code = pd_stock_list.iloc[f_index]['종목코드']
  f_name = pd_stock_list.iloc[f_index]['종목명']
  price_filename = str('price/') + str(f_code) + str('_price.csv')
  if not os.path.exists(price_filename):
    f_price = fdr.DataReader(f_code, start = start_date, end = end_date)
    f_price.to_csv(price_filename)
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 저장 완료' % (str(f_code), str(f_name)))
  else:
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 파일 존재' % (str(f_code), str(f_name)))

[종목코드 : 005930, 종목명 : 삼성전자] 가격 정보 저장 완료
[종목코드 : 000660, 종목명 : SK하이닉스] 가격 정보 저장 완료
[종목코드 : 035420, 종목명 : NAVER] 가격 정보 저장 완료
[종목코드 : 035720, 종목명 : 카카오] 가격 정보 저장 완료
[종목코드 : 207940, 종목명 : 삼성바이오로직스] 가격 정보 저장 완료
[종목코드 : 005935, 종목명 : 삼성전자우] 가격 정보 저장 완료
[종목코드 : 051910, 종목명 : LG화학] 가격 정보 저장 완료
[종목코드 : 006400, 종목명 : 삼성SDI] 가격 정보 저장 완료
[종목코드 : 005380, 종목명 : 현대차] 가격 정보 저장 완료
[종목코드 : 068270, 종목명 : 셀트리온] 가격 정보 저장 완료
[종목코드 : 000270, 종목명 : 기아] 가격 정보 저장 완료
[종목코드 : 005490, 종목명 : POSCO] 가격 정보 저장 완료
[종목코드 : 066570, 종목명 : LG전자] 가격 정보 저장 완료
[종목코드 : 028260, 종목명 : 삼성물산] 가격 정보 저장 완료
[종목코드 : 012330, 종목명 : 현대모비스] 가격 정보 저장 완료
[종목코드 : 096770, 종목명 : SK이노베이션] 가격 정보 저장 완료
[종목코드 : 051900, 종목명 : LG생활건강] 가격 정보 저장 완료
[종목코드 : 017670, 종목명 : SK텔레콤] 가격 정보 저장 완료
[종목코드 : 105560, 종목명 : KB금융] 가격 정보 저장 완료
[종목코드 : 055550, 종목명 : 신한지주] 가격 정보 저장 완료
[종목코드 : 034730, 종목명 : SK] 가격 정보 저장 완료
[종목코드 : 036570, 종목명 : 엔씨소프트] 가격 정보 저장 완료
[종목코드 : 011200, 종목명 : HMM] 가격 정보 저장 완료
[종목코드 : 015760, 종목명 : 한국전력] 가격 정보 저장 완료
[종목코드 : 032830, 종목명 : 삼성생명] 가

In [16]:
from zipfile import ZipFile
from os.path import basename

In [24]:
for foldername, subfolders, filenames in os.walk('price'):
  print(filenames)

['029780_price.csv', '060720_price.csv', '051900_price.csv', '271560_price.csv', '068240_price.csv', '336370_price.csv', '180640_price.csv', '069500_price.csv', '011210_price.csv', '092040_price.csv', '268600_price.csv', '011200_price.csv', '010780_price.csv', '214980_price.csv', '213420_price.csv', '299030_price.csv', '183300_price.csv', '015750_price.csv', '069080_price.csv', '051600_price.csv', '000990_price.csv', '016360_price.csv', '025900_price.csv', '086790_price.csv', '192820_price.csv', '036030_price.csv', '122630_price.csv', '153130_price.csv', '086900_price.csv', '110790_price.csv', '012510_price.csv', '023530_price.csv', '005250_price.csv', '043150_price.csv', '011780_price.csv', '051910_price.csv', '108320_price.csv', '013120_price.csv', '161390_price.csv', '256840_price.csv', '051905_price.csv', '001230_price.csv', '047050_price.csv', '294870_price.csv', '140410_price.csv', '009830_price.csv', '230360_price.csv', '217270_price.csv', '263750_price.csv', '036490_price.csv',

In [25]:
# Zip the files from given directory that matches the filter
def zipFilesInDir(dirName, zipFileName, filter):
   # create a ZipFile object
   with ZipFile(zipFileName, 'w') as zipObj:
       # Iterate over all the files in directory
       for folderName, subfolders, filenames in os.walk(dirName):
           for filename in filenames:
               if filter(filename):
                   # create complete filepath of file in directory
                   filePath = os.path.join(folderName, filename)
                   # Add file to zip
                   zipObj.write(filePath, basename(filePath))

In [29]:
zipFilesInDir('price', 'price.zip', lambda name : 'csv' in name)